In [1]:
import sys
import torch
sys.path.insert(0, './code/others/pytorch-i3d')
actuals_path = "actuals.npy"
predictions_path = "predictions.npy"
confusion = "confusion.npy"
training_file = "./TinyVIRAT/tiny_train.json"
test_file = "./TinyVIRAT/tiny_test.json"
classes_file = 'classes.txt'
import json
import numpy as np
from collections import defaultdict
from collections import OrderedDict
import pandas as pd
from IPython.display import display
from resizer import ResizerMainNetworkV4_3D
model_path = "code/others/pytorch-i3d/eval_models/bilinear_32_resizer_v9_final_resizer_v43r_residuals_000038.pt"
from virat_dataset import Virat



In [2]:
actuals = np.load(actuals_path)
predictions = np.load(predictions_path)
training_json = dict()
with open(training_file,'r+') as f:
    training_json = json.load(f)['tubes']
print(len((training_json)))
labels_map = dict()
reverse_map = dict()
with open(classes_file,'r+') as f:
    labels_map = {v:k for k,v in enumerate([k[:-1] for k in f.readlines()])}
    reverse_map = {v:k for k,v in labels_map.items()}
training_dataset_counts = defaultdict(int)
print(len(labels_map))
for c in training_json: 
    for l in c['label']:
        training_dataset_counts[labels_map[l]]+=1
print(training_dataset_counts)
print(reverse_map)
        


7663
26
defaultdict(<class 'int'>, {2: 32, 16: 4644, 21: 843, 12: 2561, 24: 38, 23: 118, 10: 93, 15: 1831, 3: 288, 0: 17, 6: 26, 25: 33, 13: 29, 7: 135, 18: 60, 11: 61, 22: 72, 1: 22, 4: 10, 17: 14, 19: 42, 9: 70, 20: 190, 14: 33, 8: 13, 5: 52})
{0: 'Closing', 1: 'Entering', 2: 'Exiting', 3: 'Interacts', 4: 'Loading', 5: 'Misc', 6: 'Opening', 7: 'Pull', 8: 'Push', 9: 'Riding', 10: 'Talking', 11: 'Transport_HeavyCarry', 12: 'activity_carrying', 13: 'activity_gesturing', 14: 'activity_running', 15: 'activity_standing', 16: 'activity_walking', 17: 'specialized_miscellaneous', 18: 'specialized_talking_phone', 19: 'specialized_texting_phone', 20: 'specialized_using_tool', 21: 'vehicle_moving', 22: 'vehicle_starting', 23: 'vehicle_stopping', 24: 'vehicle_turning_left', 25: 'vehicle_turning_right'}


In [3]:
correct = 0
wrong = 0
correct_map = defaultdict(int)
count_map = defaultdict(int)
wrong_map = defaultdict(int)
no_preds_map = defaultdict(int)
no_preds = 0
for k,v in zip(actuals, predictions):
    if np.all(k == v):
        correct+=1
        preds = np.where(k==1)
        for d in np.where(k==1):
            correct_map[d[0]]+=1
        
    else:
        wrong+=1
        if len(np.argwhere(v==1)) == 0:
            no_preds+=1
            for d in np.argwhere(k==0):
                no_preds_map[d[0]]+=1
    for d in np.where(k==1):
        count_map[d[0]]+=1
print(correct, wrong, no_preds)
print(len(correct_map.keys()))
print(correct_map.items())
print(count_map.items())
print(no_preds_map)
    
        


3038 2128 120
10
dict_items([(16, 1679), (21, 427), (15, 313), (12, 472), (2, 9), (3, 84), (11, 5), (9, 23), (7, 23), (10, 3)])
dict_items([(1, 21), (0, 12), (16, 2046), (21, 601), (15, 475), (12, 1321), (18, 2), (2, 19), (3, 222), (6, 11), (22, 11), (5, 19), (23, 26), (4, 3), (9, 118), (13, 2), (11, 86), (24, 5), (8, 44), (7, 42), (14, 25), (10, 52), (25, 2), (17, 1)])
defaultdict(<class 'int'>, {0: 119, 1: 114, 2: 115, 3: 108, 4: 120, 5: 120, 6: 118, 7: 119, 8: 119, 9: 117, 10: 120, 11: 119, 12: 92, 13: 120, 14: 120, 15: 80, 16: 84, 17: 120, 18: 120, 19: 120, 20: 119, 22: 111, 23: 116, 24: 117, 25: 118, 21: 87})


In [4]:
confusion_m = np.load('confusion.npy')

In [5]:
fscores = list()
for m in confusion_m:
    fs = (2*(m[1][1]))/(2*m[1][1] + m[1][0] + m[0][1])
    fscores.append(fs)

In [6]:
len(np.argwhere(np.array(fscores) == 0))

9

In [7]:
fscores_with_class_data = list()
for i,f in enumerate(fscores):
    print(f,training_dataset_counts[i], reverse_map[i], i )
    fscores_with_class_data.append((training_dataset_counts[i], f, reverse_map[i], i ))

0.0 17 Closing 0
0.0 22 Entering 1
0.4 32 Exiting 2
0.5952380952380952 288 Interacts 3
0.0 10 Loading 4
0.09090909090909091 52 Misc 5
0.0 26 Opening 6
0.6046511627906976 135 Pull 7
0.0 13 Push 8
0.6022727272727273 70 Riding 9
0.2 93 Talking 10
0.1702127659574468 61 Transport_HeavyCarry 11
0.5186104218362283 2561 activity_carrying 12
0.0 29 activity_gesturing 13
0.10526315789473684 33 activity_running 14
0.8237837837837838 1831 activity_standing 15
0.9371547992237648 4644 activity_walking 16
0.0 14 specialized_miscellaneous 17
0.0 60 specialized_talking_phone 18
0.0 42 specialized_texting_phone 19
0.6164383561643836 190 specialized_using_tool 20
0.8462664714494875 843 vehicle_moving 21
0.2682926829268293 72 vehicle_starting 22
0.5333333333333333 118 vehicle_stopping 23
0.1568627450980392 38 vehicle_turning_left 24
0.07142857142857142 33 vehicle_turning_right 25


In [8]:
print(sorted(fscores_with_class_data, key=lambda x: (x[1], x[0]) ))

[(10, 0.0, 'Loading', 4), (13, 0.0, 'Push', 8), (14, 0.0, 'specialized_miscellaneous', 17), (17, 0.0, 'Closing', 0), (22, 0.0, 'Entering', 1), (26, 0.0, 'Opening', 6), (29, 0.0, 'activity_gesturing', 13), (42, 0.0, 'specialized_texting_phone', 19), (60, 0.0, 'specialized_talking_phone', 18), (33, 0.07142857142857142, 'vehicle_turning_right', 25), (52, 0.09090909090909091, 'Misc', 5), (33, 0.10526315789473684, 'activity_running', 14), (38, 0.1568627450980392, 'vehicle_turning_left', 24), (61, 0.1702127659574468, 'Transport_HeavyCarry', 11), (93, 0.2, 'Talking', 10), (72, 0.2682926829268293, 'vehicle_starting', 22), (32, 0.4, 'Exiting', 2), (2561, 0.5186104218362283, 'activity_carrying', 12), (118, 0.5333333333333333, 'vehicle_stopping', 23), (288, 0.5952380952380952, 'Interacts', 3), (70, 0.6022727272727273, 'Riding', 9), (135, 0.6046511627906976, 'Pull', 7), (190, 0.6164383561643836, 'specialized_using_tool', 20), (1831, 0.8237837837837838, 'activity_standing', 15), (843, 0.84626647144

In [9]:
d = pd.DataFrame(sorted(fscores_with_class_data, key=lambda x: (x[1], x[0]) ), columns=['Training Counts', 'Fscore', 'Class Label', 'Class Index'])

In [10]:
d

,Training Counts,Fscore,Class Label,Class Index
0,10,0.000000,Loading,4
1,13,0.000000,Push,8
2,14,0.000000,specialized_miscellaneous,17
3,17,0.000000,Closing,0
4,22,0.000000,Entering,1
5,26,0.000000,Opening,6
6,29,0.000000,activity_gesturing,13
7,42,0.000000,specialized_texting_phone,19
8,60,0.000000,specialized_talking_phone,18
9,33,0.071429,vehicle_turning_right,25


In [167]:
val_predictions = np.load("val_predictions.npy")
val_actuals = np.load("val_actuals.npy")
logits = np.load("logits.npy")

In [206]:
reverse_map[16]

'activity_walking'

In [235]:
for k,v,l in zip(actuals, predictions,logits):
    if k[14] == 1:
        print("actual",np.argwhere(k==1))
        print("prediction", np.argwhere(v==1))
        print("logits", l[np.argwhere(k==1)])


actual [[12]
 [14]]
prediction [[12]
 [16]]
logits [[0.96948886]
 [0.00744529]]
actual [[ 5]
 [14]]
prediction [[ 5]
 [16]]
logits [[0.5268278 ]
 [0.08589473]]
actual [[14]]
prediction [[16]]
logits [[0.00035938]]
actual [[14]]
prediction [[16]]
logits [[6.3067026e-05]]
actual [[14]]
prediction [[16]]
logits [[0.01099626]]
actual [[14]]
prediction [[16]]
logits [[0.00691503]]
actual [[14]]
prediction [[16]]
logits [[0.00180467]]
actual [[14]]
prediction [[16]]
logits [[0.02460783]]
actual [[14]]
prediction [[16]]
logits [[0.07878543]]
actual [[14]]
prediction [[16]]
logits [[0.17430444]]
actual [[14]]
prediction [[14]]
logits [[0.5670985]]
actual [[14]]
prediction [[16]
 [18]]
logits [[0.469514]]
actual [[14]]
prediction [[16]
 [18]]
logits [[0.27247724]]
actual [[14]]
prediction [[14]
 [16]]
logits [[0.5259325]]
actual [[14]]
prediction [[16]
 [18]]
logits [[0.1939909]]
actual [[14]]
prediction [[14]
 [16]]
logits [[0.6767691]]
actual [[14]]
prediction [[12]
 [16]]
logits [[0.09732367

In [234]:
labels_map['activity_running']

14

In [236]:
reverse_map[16]

'activity_walking'

In [209]:
len(predictions)

5166

In [210]:
test_json = dict()
with open(test_file,'r+') as f:
    test_json = json.load(f)['tubes']
print(len((test_json)))

5166


In [212]:
for d, c in zip(test_json, actuals):
    

[[1]] [1]
[[0]] [0]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[ 0]
 [ 2]
 [15]] [2, 0, 15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[

[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[13]] [13]
[[13]] [13]
[[15]] [15]
[[15]] [15]
[[11]] [11]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[15]] [15]
[[15]] [15]
[[15]
 [16]] [15, 16]
[[15]] [15]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [13]
 [16]] [11, 13, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11]
 [16]] [11, 16]
[[11

[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16

 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]
 [18]] [18, 16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [15]] [12, 15]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16

[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[21]] [21]
[[15]] [15]
[[15]] [15]
[[23]] [23]
[[23]] [23]
[[23]] [23]
[[23]] [23]
[[23]] [23]
[[23]] [23]
[[23]] [23]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[14]] [14]
[[24]] [24]
[[24]] [24]
[[24]] [24]
[[22]] [22]
[[22]] [22]
[[22]] [22]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16

 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [17]] [17, 12]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [15]] [12, 15]
[[12]
 [14]] [14, 12]
[[12]
 [14]] [14, 12]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12

[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[12]] [12]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15]] [15]
[[15

 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[16]] [16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16]
[[12]
 [16]] [12, 16

In [232]:
model = ResizerMainNetworkV4_3D(3, 32, (112, 112), skip=False, num_resblocks=2)
model.load_state_dict(torch.load(model_path,torch.device('cpu')))
d = Virat("code/others/pytorch-i3d/TinyVIRAT", "test","classes.txt",resize=False, transforms=None,sample=False)

label map {'Closing': 0, 'Entering': 1, 'Exiting': 2, 'Interacts': 3, 'Loading': 4, 'Misc': 5, 'Opening': 6, 'Pull': 7, 'Push': 8, 'Riding': 9, 'Talking': 10, 'Transport_HeavyCarry': 11, 'activity_carrying': 12, 'activity_gesturing': 13, 'activity_running': 14, 'activity_standing': 15, 'activity_walking': 16, 'specialized_miscellaneous': 17, 'specialized_talking_phone': 18, 'specialized_texting_phone': 19, 'specialized_using_tool': 20, 'vehicle_moving': 21, 'vehicle_starting': 22, 'vehicle_stopping': 23, 'vehicle_turning_left': 24, 'vehicle_turning_right': 25}
5166 26


In [233]:
d.data

[{'path': 'code/others/pytorch-i3d/TinyVIRAT/videos/test/VIRAT_S_000205_02_000409_000566/0.mp4',
  'frames': 82,
  'label': ['Entering'],
  'dims': [82, 90, 90]},
 {'path': 'code/others/pytorch-i3d/TinyVIRAT/videos/test/VIRAT_S_000205_02_000409_000566/1.mp4',
  'frames': 52,
  'label': ['Closing'],
  'dims': [52, 74, 74]},
 {'path': 'code/others/pytorch-i3d/TinyVIRAT/videos/test/VIRAT_S_000205_02_000409_000566/2.mp4',
  'frames': 60,
  'label': ['activity_walking'],
  'dims': [60, 60, 60]},
 {'path': 'code/others/pytorch-i3d/TinyVIRAT/videos/test/VIRAT_S_000205_02_000409_000566/3.mp4',
  'frames': 140,
  'label': ['activity_walking'],
  'dims': [140, 68, 68]},
 {'path': 'code/others/pytorch-i3d/TinyVIRAT/videos/test/VIRAT_S_000205_02_000409_000566/4.mp4',
  'frames': 105,
  'label': ['activity_walking'],
  'dims': [105, 72, 72]},
 {'path': 'code/others/pytorch-i3d/TinyVIRAT/videos/test/VIRAT_S_000205_02_000409_000566/5.mp4',
  'frames': 74,
  'label': ['activity_walking'],
  'dims': [7